In [105]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 100

In [106]:
# ディレクトリ 内のメールファイルを読み込む

from fileRetrieval import path_list

directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=path_list(directory_path)

In [107]:
# テキストファイルをデータフレームに格納する

from fileToDataFrame import fileToDataFrame

df = fileToDataFrame(file_list)

In [108]:
from toSentences import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.05260896682739258 s


In [109]:
# トリプルテーブルを作る

from tripleExtraction import tripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

counter = 0
for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(triples))
    triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(triples))
    triple_dict['Triple'].extend(triples)
    
triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')
        
end = time.time()
print(end-start,'s')

198.30864787101746 s


In [115]:
pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']).to_csv('output/allen_p_inbox_triple.csv')

In [116]:
triple_df.head()

,Message-ID,Sentence-ID,Triple
0,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(TVA Tennessee Valley Author= ity (, takes, lead)"
1,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(), took, the top spot)"
2,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(dela= ys sector, restructuring, on foreign gas)"
3,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(ASPs, allow, companies to focus Customer service function a ke= y attraction [ IMAGE ] [ IMAGE ..."
4,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(companies, focus, Customer service function ] [ IMAGE ] [ IMAGE ] [)"


In [117]:
sentence_df.head()

,Message-ID,Sentence-ID,Sentence
0,<22534132.1075858645229.JavaMail.evans@thyme>,0,=20 [IMAGE]=09 [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [I= MAGE] [IMAGE]...
1,<22534132.1075858645229.JavaMail.evans@thyme>,1,Coalition aims to sell residential green power in R.I. fu= ll story... Enron's debt ratings cut ...
2,<22534132.1075858645229.JavaMail.evans@thyme>,2,Domestic pipeline capacity must grow to meet expected g= as demand full story... CMS could expan...
3,<22534132.1075858645229.JavaMail.evans@thyme>,3,Ga= s (Henry) 3.202 0.161 5.29% Propane (Nov) 39.80 0.20 0.51% Palo Verde (No= v) 32.25 0.00 0.0...
4,<21920205.1075855377891.JavaMail.evans@thyme>,0,This request has been pending your approval for 60 days.


In [118]:
df.head()

,Path,Message-ID,Date,From,To,Cc,Bcc,X-From,X-To,X-cc,X-bcc,Subject,Message
0,enron/maildir/allen-p/inbox/36.,<22534132.1075858645229.JavaMail.evans@thyme>,"Mon, 29 Oct 2001 15:48:08 -0800 (PST)",ei_editor@ftenergy.com,einsighthtml@listserv.ftenergy.com,None,None,Energy Insight Editor <EI_editor@FTENERGY.COM>@ENRON,EINSIGHTHTML@LISTSERV.FTENERGY.COM,,,"TVA takes lead in sales, generation by utilities",=20\n[IMAGE]=09\n\n\n [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [IMAGE] [...
1,enron/maildir/allen-p/inbox/19.,<21920205.1075855377891.JavaMail.evans@thyme>,"Tue, 1 Jan 2002 17:19:40 -0800 (PST)",arsystem@mailman.enron.com,k..allen@enron.com,None,None,ARSystem <ARSystem@mailman.enron.com>@ENRON,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PALLEN>",,,Your Approval is Overdue: Access Request for matt.smith@enron.com,This request has been pending your approval for 60 days. Please click http://itcapps.corp.enro...
2,enron/maildir/allen-p/inbox/3.,<10326858.1075855377484.JavaMail.evans@thyme>,"Mon, 10 Dec 2001 15:31:51 -0800 (PST)",david.port@enron.com,k..allen@enron.com,None,None,"Port, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=DPORT>","Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Pallen>",,,FW: Gas P&L by day,"Phillip\nMy interpretation of this is that we made $1.2Bn total, half from new deals and the oth..."
3,enron/maildir/allen-p/inbox/9.,<32169781.1075855377639.JavaMail.evans@thyme>,"Sun, 30 Dec 2001 18:33:29 -0800 (PST)",unsubscribe-i@networkpromotion.com,pallen@enron.com,None,None,itsImazing <unsubscribe-i@networkpromotion.com>@ENRON,PALLEN@ENRON.COM,,,"Too many to chose from - CD player, 2 way Radios, Pencam....","[IMAGE]\nGet your FREE* Reward NOW!\n=09=09[IMAGE]=09\n=09PHILLIP, this is for REAL... one of Am..."
4,enron/maildir/allen-p/inbox/13.,<7618763.1075855377753.JavaMail.evans@thyme>,"Mon, 31 Dec 2001 10:53:43 -0800 (PST)",louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron.com, rob.milnthorp@enron.com, \n\tjohn.zufferli@e...",john.lavorato@enron.com,john.lavorato@enron.com,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/CN=LKITCHEN>","Colwell, Wes </O=ENRON/OU=NA/CN=RECIPIENTS/CN=WCOLWEL>, Hodges, Georgeanne </O=ENRON/OU=NA/CN=RE...","Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JLAVORA>",,NETCO,The New Year has arrived and we really to finalize a lot of the work with regards to moving into...
